<a href="https://colab.research.google.com/github/JLMuehlbauer/food-sales-prediction/blob/main/Project_1_food_sales_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Food Sales Predictions
---
#Project 1: Part 1
Jackson Muehlbauer

In [55]:
import pandas as pd

##Loading Data

In [56]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [57]:
#Reading data
filepath = '/content/drive/My Drive/Colab Notebooks/2: Week 2 Pandas/sales_predictions.csv'
df = pd.read_csv(filepath)
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


##Data Cleaning

### 1. Number of (Rows, Columns)

In [59]:
# 1. Number of (Rows, Columns)
df.shape

(8523, 12)

### 2. What are the datatypes in each column?

In [60]:
# 2. What are the datatypes in each column?
df.dtypes 

Item_Identifier               object
Item_Weight                  float64
Item_Fat_Content              object
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
Outlet_Size                   object
Outlet_Location_Type          object
Outlet_Type                   object
Item_Outlet_Sales            float64
dtype: object

### 3. Are there duplicated rows? If so, drop them

In [61]:
# 3. Are there duplicate rows? If so, drop them
df.duplicated().value_counts()

False    8523
dtype: int64

There are no duplicated rows

### 4. Are there missing values?

In [62]:
# 4. Are there missing values?
df.isna().sum()


Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [63]:
# Understanding Outlet_Size values
print(df['Outlet_Size'].value_counts())
print(f"NaN {df['Outlet_Size'].isna().sum()}")

Medium    2793
Small     2388
High       932
Name: Outlet_Size, dtype: int64
NaN 2410


### 5. Addressing missing values
---
Item_Weight and Outlet_Size both have missing values...

1. Item_Weight is of dtype float. I assume that it could be a useful column for predictions. The column has around 18% missing values, more than I would want to drop. I will impute the mean in place of the missing values. 

2. Outlet_Size is categorical data (with only 3 unique values) of type string. I also assume this could be a valuable column for predictions. There may also be some key learnings that can be made by keeping this data, thus, I will replace missing values with "missing"

In [64]:
# Filter for missing values
weight_missing = df['Item_Weight'].isna()
mean_weight = df['Item_Weight'].mean()
df.loc[weight_missing, 'Item_Weight'] = mean_weight

# Checking
df['Item_Weight'].isna().sum()

0

In [65]:
# Filling NaN values in Outlet_Size with 'missing'
df['Outlet_Size'].fillna('missing', inplace = True)

# Checking
df['Outlet_Size'].isna().sum()

0

### 6. Confirm there are no missing values

In [66]:
# Confirming that there are no longer missing values
df.isna().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64

### 7) Find and fix any inconsistent categories of data (example: fix cat, Cat, and cats so that they are consistent).


In [67]:
print(df.nunique())
print(df.dtypes)

Item_Identifier              1559
Item_Weight                   416
Item_Fat_Content                5
Item_Visibility              7880
Item_Type                      16
Item_MRP                     5938
Outlet_Identifier              10
Outlet_Establishment_Year       9
Outlet_Size                     4
Outlet_Location_Type            3
Outlet_Type                     4
Item_Outlet_Sales            3493
dtype: int64
Item_Identifier               object
Item_Weight                  float64
Item_Fat_Content              object
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
Outlet_Size                   object
Outlet_Location_Type          object
Outlet_Type                   object
Item_Outlet_Sales            float64
dtype: object


In [68]:
#checking Item_Fat_Content for inconsistencies
df['Item_Fat_Content'].value_counts()

Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64

1. I assume that 'Low Fat', 'LF', and 'low fat' are all the same category. Thus, I wils change all values to Low Fat
2. I assume 'Regular' and 'reg' are the same, thus, I will change all values to 'Regular'

In [69]:
# making filters
low_fat = (df['Item_Fat_Content'] == 'LF') | (df['Item_Fat_Content'] == 'low fat')
reg_fat = df['Item_Fat_Content'] == 'reg'

In [70]:
# Using .loc to apply these filters to change the values
df.loc[low_fat, 'Item_Fat_Content'] = 'Low Fat'
df.loc[reg_fat, 'Item_Fat_Content'] = 'Regular'

#Checking that the changes were made
df['Item_Fat_Content'].value_counts()

Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64

In [71]:
#checking Item_Type for inconsistencies
print(df['Item_Type'].value_counts())
print('\n')
print(df['Outlet_Identifier'].value_counts())
print('\n')
print(df['Outlet_Size'].value_counts())
print('\n')
print(df['Outlet_Location_Type'].value_counts())
print('\n')
print(df['Outlet_Type'].value_counts())
#checks out

Fruits and Vegetables    1232
Snack Foods              1200
Household                 910
Frozen Foods              856
Dairy                     682
Canned                    649
Baking Goods              648
Health and Hygiene        520
Soft Drinks               445
Meat                      425
Breads                    251
Hard Drinks               214
Others                    169
Starchy Foods             148
Breakfast                 110
Seafood                    64
Name: Item_Type, dtype: int64


OUT027    935
OUT013    932
OUT049    930
OUT046    930
OUT035    930
OUT045    929
OUT018    928
OUT017    926
OUT010    555
OUT019    528
Name: Outlet_Identifier, dtype: int64


Medium     2793
missing    2410
Small      2388
High        932
Name: Outlet_Size, dtype: int64


Tier 3    3350
Tier 2    2785
Tier 1    2388
Name: Outlet_Location_Type, dtype: int64


Supermarket Type1    5577
Grocery Store        1083
Supermarket Type3     935
Supermarket Type2     928
Name: Outlet_Type,

In [72]:
# If this data is used for a prediction model, the Item_Identifier appears to have far to many unique values to be useful. 
# However, it can't be used as an indexer because it is not unique (duplicates exist)
df.drop(columns = 'Item_Identifier', inplace = True)
df.head()

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,missing,Tier 3,Grocery Store,732.3800
4,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


### 8. For any numerical columns, obtain the summary statistics of each (min, max, mean)


In [73]:
# Summary statistics
df.describe(exclude=[object])

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,8523.000000,8523.000000,8523.000000,8523.000000,8523.000000
mean,12.857645,0.066132,140.992782,1997.831867,2181.288914
std,4.226124,0.051598,62.275067,8.371760,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,9.310000,0.026989,93.826500,1987.000000,834.247400
50%,12.857645,0.053931,143.012800,1999.000000,1794.331000
75%,16.000000,0.094585,185.643700,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


In [75]:
# Only showing min, max, mean
print(f'Minimum value: \n{df.min(numeric_only = True)}\n')
print(f'Maximum value: \n{df.max(numeric_only = True)}\n')
print(f'Mean value: \n{df.mean(numeric_only = True)}\n')


Minimum value: 
Item_Weight                     4.555
Item_Visibility                 0.000
Item_MRP                       31.290
Outlet_Establishment_Year    1985.000
Item_Outlet_Sales              33.290
dtype: float64

Maximum value: 
Item_Weight                     21.350000
Item_Visibility                  0.328391
Item_MRP                       266.888400
Outlet_Establishment_Year     2009.000000
Item_Outlet_Sales            13086.964800
dtype: float64

Mean value: 
Item_Weight                    12.857645
Item_Visibility                 0.066132
Item_MRP                      140.992782
Outlet_Establishment_Year    1997.831867
Item_Outlet_Sales            2181.288914
dtype: float64



##Exploratory Visuals

##Explanatory Visuals